In [ ]:
from google import genai
import os
from google.genai import types

In [ ]:
client = genai.Client(
      vertexai=True
  )


In [ ]:
client

In [ ]:
client = genai.Client()
response = client.models.generate_content(
    model="gemini-2.0-flash", contents="Is it too late to join the course?"
)
print(response.text)

In [ ]:
#!wget https://raw.githubusercontent.com/alexeygrigorev/minsearch/refs/heads/main/minsearch.py

In [ ]:
import minsearch

In [ ]:
minsearch

In [ ]:
#!wget https://raw.githubusercontent.com/DataTalksClub/llm-zoomcamp/refs/heads/main/01-intro/documents.json

In [ ]:
import json

In [ ]:
with open('documents.json', 'rt') as f_in:
    docs_raw  = json.load(f_in)

In [ ]:
documents = []

for course_dict in docs_raw:
    for doc in course_dict['documents']:
        doc['course'] = course_dict['course']
        documents.append(doc)

In [ ]:
documents[0]

In [ ]:
index = minsearch.Index(
    text_fields = ["question", "text", "section"],
    keyword_fields=["course"]
)

In [ ]:
index.fit(documents)

In [ ]:
q = "The course has already started. Can I still enroll?"

In [ ]:
boost = {"question": 3.0, "section": 0.5}

results = index.search(
    query=q,
    filter_dict={'course':'data-engineering-zoomcamp'},
    boost_dict=boost,
    num_results=5
)

In [ ]:
client

In [ ]:
response = client.models.generate_content(
    model="gemini-2.0-flash", 
    contents=q
)
print(response.text)

In [ ]:
prompt_template = """
You're a course teaching assistant. Answer the QUESTION based on the CONTEXT from the FAQ database. 
Use only the facts from the CONTEXT when answering the QUESTION.
If the CONTEXT doesn't contain the answer, output NONE.

QUESTION: {question}

CONTEXT: 
{context}
""".strip()

In [ ]:
context = ""
for doc in results:
    context = context + f"section: {doc['section']}\nquestion: {doc['question']}\nanswer: {doc['text']}\n\n"

In [ ]:
prompt = prompt_template.format(question=q, context=context).strip()

In [ ]:
print(prompt)

In [ ]:
response = client.models.generate_content(
    model="gemini-2.0-flash", 
    contents=prompt
)
print(response.text)

In [ ]:
def search(query):
    boost = {"question": 3.0, "section": 0.5}

    results = index.search(
        query=query,
        filter_dict={'course':'data-engineering-zoomcamp'},
        boost_dict=boost,
        num_results=5
    )
    return results

In [ ]:
search('how do i run kafka?')

In [ ]:
def build_prompt(query, search_results):
    prompt_template = """
        You're a course teaching assistant. Answer the QUESTION based on the CONTEXT from the FAQ database. 
        Use only the facts from the CONTEXT when answering the QUESTION.
        If the CONTEXT doesn't contain the answer, output NONE.
        
        QUESTION: {question}
        
        CONTEXT: 
        {context}
        """.strip()
    context = ""
    for doc in search_results:
        context = context + f"section: {doc['section']}\nquestion: {doc['question']}\nanswer: {doc['text']}\n\n"

    prompt = prompt_template.format(question=query, context=context).strip()
    return prompt

In [ ]:
def llm(prompt):
    response = client.models.generate_content(
        model="gemini-2.0-flash", 
        contents=prompt
    )
    return response.text

In [ ]:
def rag(query):
    search_results = search(query)
    prompt = build_prompt(query, search_results)
    answer = llm(prompt)
    return answer

In [ ]:
print(rag('how do I run kafka?'))

In [ ]:
documents[0]

In [ ]:
from elasticsearch import Elasticsearch

In [94]:
es_client = Elasticsearch('http://localhost:9200')

In [95]:
es_client.info()

ObjectApiResponse({'name': '7b385051d6db', 'cluster_name': 'docker-cluster', 'cluster_uuid': '0Wnl4O2hSFexy0_4Q1m43A', 'version': {'number': '8.4.3', 'build_flavor': 'default', 'build_type': 'docker', 'build_hash': '42f05b9372a9a4a470db3b52817899b99a76ee73', 'build_date': '2022-10-04T07:17:24.662462378Z', 'build_snapshot': False, 'lucene_version': '9.3.0', 'minimum_wire_compatibility_version': '7.17.0', 'minimum_index_compatibility_version': '7.0.0'}, 'tagline': 'You Know, for Search'})

In [96]:
index_settings = {
    "settings": {
        "number_of_shards": 1,
        "number_of_replicas": 0
    },
    "mappings": {
        "properties": {
            "text": {"type": "text"},
            "section": {"type": "text"},
            "question": {"type": "text"},
            "course": {"type": "keyword"} 
        }
    }
}

In [ ]:
index_name = "course-questions"

es_client.indices.create(index=index_name, settings=index_settings['settings'], mappings=index_settings['mappings'])

ObjectApiResponse({'acknowledged': True, 'shards_acknowledged': True, 'index': 'course-questions2'})

In [80]:
#!pip install tqdm
from tqdm.auto import tqdm

In [99]:
for doc in tqdm(documents):
    es_client.index(index=index_name, document=doc)

100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 948/948 [00:02<00:00, 382.05it/s]


In [100]:
query = q

In [ ]:
result_docs

In [ ]:
def rag(query):
    search_results = elastic_search(query)
    prompt = build_prompt(query, search_results)
    answer = llm(prompt)
    return answer

In [65]:
rag('how do i run kafka?')

'NONE\n'

In [ ]:
\

In [ ]:
[]